## Adding Elevation Data from USGS DEM to OSM
---
Method based on [Liu et al. 2018](https://doi.org/10.1016/j.trc.2018.05.004)

In [9]:
import geopandas as gpd
import contextily as cx
import pandas as pd
import rasterio
from rasterio.features import geometry_mask
from rasterio.plot import show
import numpy as np
from shapely.geometry import box, mapping
from shapely.ops import Point
from tqdm import tqdm
from shapely.ops import LineString
import pickle

from bikewaysim.paths import config
from bikewaysim.network import elevation_tools

### Get DEM TIFF download links from [USGS.gov](https://apps.nationalmap.gov/downloader/#/?z=4&y=37.99999999999999&x=-95&basemap=usgs_topo&datasets=&layerIds=)

In [10]:
links = gpd.read_file(config['network_fp'] / "osm.gpkg",layer="raw")
links.set_index('osmid',inplace=True)
links.to_crs('epsg:4326',inplace=True)
dem_urls, response = elevation_tools.get_dem_urls(links)
print(len(dem_urls),'TIFF files for the provided network')
print("Names:",set([dem_url.split('/')[-3] for dem_url in dem_urls]))

4 TIFF files for the provided network
Names: {'GA_Statewide_2018_B18_DRRA'}


### Choose which product to use from the names (usually pick the one that was uploaded last)

In [11]:
# for Georgia, it'd be GA_Statewide
dem_urls = [url for url in dem_urls if 'GA_Statewide' in url]

#if you want to download the full DEM files run this
#elevation_tools.download_dem(dem_urls,config['usgs_fp'])

### Project Network to Same CRS as TIFF

In [12]:
dem_crs = rasterio.open(dem_urls[0]).crs
links.to_crs(dem_crs,inplace=True)

#same dem as text for future reference
with (config['network_fp'] / 'dem_crs.txt').open('w') as file:
    file.write(dem_crs.to_string())

### Sampling Settings (Default is 10m)
If a link is shorter than this, only the start and end point will be used. Last point will always be included in the profile (e.g., link with length 11 m will be sampled at 0, 10, and 11 meters)

In [13]:
interpolate_dist_m = 10
print((links.length < interpolate_dist_m).sum(),'links of',links.shape[0] ,'total were less than',interpolate_dist_m,'meters')
print(links[links.length<interpolate_dist_m].length.sum().round(0),'of',links.length.sum().round(0),'meters',round(links[links.length<interpolate_dist_m].length.sum() / links.length.sum() * 100,0),'%')

3018 links of 15120 total were less than 10 meters
16026.0 of 1163493.0 meters 1.0 %


In [14]:
interpolated_points_dict = elevation_tools.interpolate_points(links,interpolate_dist_m)

### Sample values from DEM (17-25 minutes for study area)
Replace the nan values on the interpolated points.

In [15]:
for dem_url in tqdm(dem_urls):
    elevation_tools.sample_elevation(dem_url,links,interpolated_points_dict)

100%|██████████| 4/4 [01:12<00:00, 18.04s/it]


### Export sampled interpolated points

In [16]:
with (config['network_fp']/'elevation.pkl').open('wb') as fh:
    pickle.dump(interpolated_points_dict,fh)